In [1]:
! mkdir -p data/code

# 下载词典数据

In [2]:
import json, os, random, time, math, multiprocessing, traceback
import boto3
import sagemaker
from sagemaker import image_uris, get_execution_role
from sagemaker.session import production_variant
from pyarrow import parquet
from datetime import datetime, timedelta
import pickle

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
s3_cli = boto3.client('s3')
sm_sess = sagemaker.Session()
print('aws region:', sm_sess.boto_region_name)
sm_cli = boto3.client('sagemaker')
role = get_execution_role()
print('role:', role)

aws region: ap-southeast-1
role: arn:aws:iam::580243821396:role/service-role/AmazonSageMaker-ExecutionRole-20241030T174232


In [4]:
def get_s3_file_list_by_prefix(bucket, prefix, max_cnt=int(1e6)):
        file_list = sorted([v['Key'] for v in s3_cli.list_objects(Bucket=bucket,
                                                                  Prefix=prefix, MaxKeys=max_cnt).get('Contents', [])])
        print('List S3 %d files in prefix: %s' % (len(file_list), prefix))
        return file_list

def get_latest_dict_path(bucket, prefix):
    file_list_ = get_s3_file_list_by_prefix(bucket, prefix)
    file_list_ = [(item, item.split("ds=")[1][:8]) for item in file_list_]
    print(file_list_)
    latest_date = sorted(file_list_, key=lambda item:item[1], reverse=True)[0][1]
    return latest_date

In [5]:
latest_date = get_latest_dict_path("warehouse-algo", "cn_all_goods_info")
yesterday = latest_date
# yesterday_lines = save_file_from_s3('warehouse-algo', 'cn_all_goods_info/', days_range=[yesterday],
#                                         fmt='parquet')

List S3 167 files in prefix: cn_all_goods_info
[('cn_all_goods_info/ds=20240603/part-00000-0f9cd02d-f31e-4165-bac2-d2c5e68d703b-c000', '20240603'), ('cn_all_goods_info/ds=20240604/part-00000-0eaddece-0104-4dbb-b79b-618e81aab607-c000', '20240604'), ('cn_all_goods_info/ds=20240605/part-00000-5a697e10-02f7-45bb-a61a-724f1b2faf8d-c000', '20240605'), ('cn_all_goods_info/ds=20240606/part-00000-360d4e6c-8b8b-4962-9188-e8ee937be75c-c000', '20240606'), ('cn_all_goods_info/ds=20240607/part-00000-635083ef-3539-44b2-9b9b-684c9d56ec7b-c000', '20240607'), ('cn_all_goods_info/ds=20240608/part-00000-3684794a-d12e-4c8f-9eb3-e5200afe6225-c000', '20240608'), ('cn_all_goods_info/ds=20240609/part-00000-dd661ec6-f914-4b9f-8460-08fe888a0c22-c000', '20240609'), ('cn_all_goods_info/ds=20240610/part-00000-e716a8e1-f19c-46c7-a12d-a16df95a9baf-c000', '20240610'), ('cn_all_goods_info/ds=20240611/part-00000-a89e31b1-3486-4600-a390-7611f752cdd9-c000', '20240611'), ('cn_all_goods_info/ds=20240612/part-00000-27ce48d3-

In [6]:
latest_date

'20241117'

# download goods attrs dict

In [7]:
def get_goods_prop_values(yesterday):
    selected_fields = ['belt', 'bra type', 'chest pad', 'cid1_name', 'cid2_name', 'cid3_name', 'cid4_name',
                       'collection', 'colors', 'cup shape', 'decoration(accessories)', 'decoration(bags)',
                       'decoration(jewelry)', 'design detail', 'features', 'fit', 'goods_desc', 'hem shaped', 'length',
                       'lining', 'main fabric', 'main material', 'main material (bag)', 'material (beauty)',
                       'material(glasses)', 'neckline', 'occasion', 'opening(bags )', 'other material', 'pattern',
                       'pattern(glasses)', 'season', 'shape', 'silhouette', 'sleeve length', 'sleeve type', 'softness',
                       'strap type', 'strap type(swimwear)', 'style', 'style(home)', 'title', 'type', 'underwire',
                       'waistline']

    def parquet_iter(parquet_file):
        f = parquet.ParquetFile(parquet_file)
        for batch in f.iter_batches():
            for msg in batch.to_pylist():
                yield msg
        f.close()

    def load_record(tmp_file, fmt):
        if fmt == 'parquet':
            for v in parquet_iter(tmp_file):
                yield v
        elif fmt == 'lines':
            with open(tmp_file, 'rt') as f:
                for v in f.readlines():
                    yield v
        elif fmt == 'csv':
            with open(tmp_file, 'rt') as f:
                header = {name: idx for idx, name in enumerate(f.readline().strip('\n').split('\t'))}
                for line in f:
                    items = line.strip('\n').split('\t')
                    yield {k: items[v] for k, v in header.items()}

    def load_s3_dir(bucket, prefix, days_range, fmt='parquet', save_path="./data/sort/tmp.txt"):
        days_range = {k: 1 for k in days_range}
        paginator = s3_cli.get_paginator('list_objects_v2')
        page_iter = paginator.paginate(Bucket=bucket, Prefix=prefix)
        file_list = sum([[v['Key'] for v in page.get('Contents', [])] for page in page_iter], [])

        def get_file_ds(filename):
            if "ds=" not in filename:
                return ""
            else:
                ds = filename.split("ds=")[1].split("/")[0]
                return ds

        if days_range is not None:
            file_list = [v for v in file_list if get_file_ds(v) in days_range]

        print(file_list)

        index = 0
        for key in file_list:
            success = False
            for _ in range(10):
                # print('Down', key, save_path)
                s3_cli.download_file(Bucket=bucket, Key=key, Filename=save_path)
                success = True
                break

            if success:
                # print('Load', key, save_path)
                for record in load_record(save_path, fmt):
                    yield record
                    index += 1
                    if not index % 100000:
                        print('Loading ', index, 'records', end=', ')
                        # print('Finished loading from:', prefix)

    def save_file_from_s3(bucket, prefix, days_range, fmt='parquet', save_path="./tmp.txt"):
        ret = load_s3_dir(bucket, prefix, days_range, fmt, save_path)
        ret = list(ret)
        print("the length of result is", len(ret))
        if os.path.exists(save_path):
            os.remove(save_path)
        return ret

    yesterday_lines = save_file_from_s3(bucket='warehouse-algo', prefix='cn_all_goods_info', days_range=[yesterday],
                                        fmt='parquet')
    goods_id2prop_values = {}
    goods_id2features = {}
    for line in yesterday_lines:
        goods_id = line["goods_id"]

        prop_map = line["props_map"].lower()
        prop_map = json.loads(prop_map)
        title = line["title"]
        cid1 = line["cid1"]
        cid2 = line["cid2"]
        cid3 = line["cid3"]
        cid4 = line["cid4"]
        attrs = []
        for k, v in prop_map.items():
            if k.strip() == "" or v.strip() == "":
                continue
            if k not in selected_fields:
                continue
            v = v.replace(" ", "-")
            k = k.replace(" ", "-")
            attrs.append(k + "::" + v)
        attrs = " ".join(attrs)
        goods_id2prop_values[str(goods_id)] = attrs
        features = {"title": title, "cid1": cid1, "cid2": cid2, "cid3": cid3, "cid4": cid4, "attrs": attrs}
        goods_id2features[str(goods_id)] = features
    return goods_id2features

In [8]:
goods_id2features = get_goods_prop_values(latest_date)
with open("./data/goods_id2features.pkl", "wb") as f:
    pickle.dump(goods_id2features, f)

['cn_all_goods_info/ds=20241117/part-00000-93d46cf5-2222-441e-9c55-2ee18f4e346a-c000']
Loading  100000 records, the length of result is 145871


In [9]:
%%writefile data/code/inference.py

# coding: utf-8

import json
import logging
import pickle
import os

logging.error('[DEBUG] current dir: %s %s', os.getcwd(), os.listdir("/opt/ml/model/"))

with open("/opt/ml/model/goods_id2features.pkl", "rb") as f:
    goods_id2features = pickle.load(f)


def transform_inputs(data):
    d = data.read()
    d = json.loads(d)
    d = d["inputs"]
    """
    
    country: string
    high_level_seq: ["goods_id1", "goods_id2", ...] 20 不足传 ','
    low_level_seq: ["goods_id1", "goods_id2", ...]20 不足传 ','

    title: ["title1", "title2", ...]
    cid1: ["cid1-1", "cid1-2", ...]
    cid2: ["cid2-1", "cid2-2", ...]
    cid3: ["cid3-1", "cid3-2", ...]
    cid4: ["cid4-1", "cid4-2", ...]
    uuid: string
    goods_id: [goods_id1, goods_id2, ]
    query: string
    query_seg: [word1, word2]
    title_seg: [[word1, word2, ], [word1, word2, ],...]
    
    """
    new_d = {}
    goods_num = len(d["goods_id"])
    high_level_seq = d["high_level_seq"]
    low_level_seq = d["low_level_seq"]
    query_seg = d["query_seg"]
    title = d["title"]
    cid3 = d["cid3"]
    cid4 = d["cid4"]
    cid1 = d["cid1"]
    cid2 = d["cid2"]
    title_seg = d["title_seg"]
    goods_id = d["goods_id"]

    retarget = []
    attrs = []

    for i, good_id in enumerate(goods_id):
        # logging.error('[DEBUG] good_id: %s %s %s %s', i, good_id, cur_high_level_seq, cur_low_level_seq)
        if str(good_id) in high_level_seq or str(good_id) in low_level_seq:
            retarget.append([1])
        else:
            retarget.append([0])

        if str(good_id) in goods_id2features:
            cur_attrs = goods_id2features[str(good_id)]["attrs"].split()
            if len(cur_attrs) <= 20:
                cur_attrs = cur_attrs + [","] * (20 - len(cur_attrs))
            else:
                cur_attrs = cur_attrs[:20]
            attrs.append(cur_attrs)
        else:
            attrs.append([","] * 20)

    new_d["country"] = [[d["country"]] for _ in range(goods_num)]
    new_d["high_level_seq"] = [high_level_seq for _ in range(goods_num)]
    new_d["low_level_seq"] = [low_level_seq for _ in range(goods_num)]
    new_d["query_seg"] = [query_seg for _ in range(goods_num)]
    new_d["title"] = [[item] for item in title]
    new_d["query_seg_cid3"] = [[query_seg_word + ",," + str(cur_cid3) for query_seg_word in query_seg] for cur_cid3 in
                               cid3]
    new_d["uuid"] = [[d["uuid"]] for _ in range(goods_num)]
    new_d["query_seg_cid4"] = [[query_seg_word + ",," + str(cur_cid4) for query_seg_word in query_seg] for cur_cid4 in
                               cid4]
    new_d["query_seg_cid1"] = [[query_seg_word + ",," + str(cur_cid1) for query_seg_word in query_seg] for cur_cid1 in
                               cid1]
    new_d["query_seg_cid2"] = [[query_seg_word + ",," + str(cur_cid2) for query_seg_word in query_seg] for cur_cid2 in
                               cid2]
    new_d["title_seg"] = title_seg
    new_d["cid4"] = [[cur_cid4] for cur_cid4 in cid4]
    new_d["country_goods_id"] = [[d["country"] + ",," + str(good_id)] for good_id in goods_id]
    new_d["cid2"] = [[cur_cid2] for cur_cid2 in cid2]
    new_d["cid3"] = [[cur_cid3] for cur_cid3 in cid3]
    new_d["cid1"] = [[cur_cid1] for cur_cid1 in cid1]
    new_d["query_goods_id"] = [[d["query"] + ",," + str(good_id)] for good_id in goods_id]
    new_d["query_seg_goods_id"] = [[query_seg_word + ",," + str(good_id) for query_seg_word in query_seg] for good_id in
                                   goods_id]
    new_d["query"] = [[d["query"]] for _ in range(goods_num)]
    new_d["goods_id"] = [[good_id] for good_id in goods_id]
    new_d["query_cid1"] = [[d["query"] + ",," + str(cur_cid1)] for cur_cid1 in cid1]
    new_d["country_cid4"] = [[d["country"] + ",," + str(cur_cid4)] for cur_cid4 in cid4]
    new_d["query_cid2"] = [[d["query"] + ",," + str(cur_cid2)] for cur_cid2 in cid2]
    new_d["country_cid3"] = [[d["country"] + ",," + str(cur_cid3)] for cur_cid3 in cid3]
    new_d["country_cid2"] = [[d["country"] + ",," + str(cur_cid2)] for cur_cid2 in cid2]
    new_d["country_cid1"] = [[d["country"] + ",," + str(cur_cid1)] for cur_cid1 in cid1]
    new_d["query_cid3"] = [[d["query"] + ",," + str(cur_cid3)] for cur_cid3 in cid3]
    new_d["query_cid4"] = [[d["query"] + ",," + str(cur_cid4)] for cur_cid4 in cid4]
    new_d["retarget"] = retarget
    new_d["attrs"] = attrs

    new_d = {"inputs": new_d}
    new_d = json.dumps(new_d).encode('utf-8')
    return new_d if len(new_d) else ''


def input_handler(data, context):
    """Pre-process request input before it is sent to TensorFlow Serving REST API
    Args:
        data (obj): the request data stream
        context (Context): an object containing request and configuration details
    Returns:
        (dict): a JSON-serializable dict that contains request body and headers
    """

    if context.request_content_type == "application/json":
        # d = data.read().decode('utf-8')
        d = transform_inputs(data)
        # logging.error('[DEBUG] request_data: %s %s', context.request_content_type, d)
        return d if len(d) else ''

    raise ValueError('Invalid req type: %s' % context.request_content_type)


def output_handler(data, context):
    """Post-process TensorFlow Serving output before it is returned to the client.
    Args:
        data (obj): the TensorFlow serving response
        context (Context): an object containing request and configuration details
    Returns:
        (bytes, string): data to return to client, response content type
    """
    logging.error('[DEBUG] output_data: %s %s', type(data), data)
    if data.status_code != 200:
        raise ValueError(data.content.decode('utf-8'))

    response_content_type = context.accept_header
    prediction = data.content
    return prediction, response_content_type



Overwriting data/code/inference.py


In [10]:
model_data_pattern = "rec_laidehe/test/all-ctr-test/model/ds=%s/model.tar.gz"
# s3://warehouse-algo/rec_laidehe/test/all-ctr-test/model/ds=20241014/



def wait_for_ready(target_date, wait_window=3600 * 24 * 3, wait_interval=60 * 5):
    begin = time.time()
    model_data = model_data_pattern % target_date
    model_data_done = model_data + '.done'
    while True:
        now = time.time()
        if now - begin > wait_window:
            # alert('Model of %s done %s DO NOT exists after %.2f hours, maybe something is wrong' % (
            #     target_date, model_data_done, (now - begin) / 3600))
            raise
        model_list = get_s3_file_list_by_prefix("warehouse-algo", model_data)
        if len(model_list) < 2:
            print('Model of %s done %s DO NOT exists, %d file(s), continue wait after %.2f hours' % (
                target_date, model_data_done, len(model_list), (now - begin) / 3600))
            time.sleep(wait_interval)
            continue
        elif len(model_list) > 2:
            # alert('Too many Model files of %s, %d files, something is wrong' % (target_date, len(model_list)))
            raise
        elif model_list != [model_data, model_data_done]:
            # alert('Model files of %s wrong: %s' % (target_date, model_list))
            raise

        print('Model of %s is ready now, start processing.' % target_date)
        return model_data
model_data = model_data_pattern % '20241014'


In [11]:
# 等太久
# current_date = datetime.now().date()
# current_date = current_date - timedelta(days=0)
# yesterday = current_date - timedelta(days=1)
# yesterday = yesterday.strftime("%Y%m%d")
# yesterday='20241014'
# model_data = wait_for_ready(yesterday, wait_window=3600 * 24 * 3, wait_interval=60 * 5)
# # model_data = wait_for_ready("20240612", wait_window=3600 * 24 * 3, wait_interval=60 * 5)

In [12]:
s3_model_tar_file = "s3://warehouse-algo/"+model_data

In [13]:
s3_model_tar_file

's3://warehouse-algo/rec_laidehe/test/all-ctr-test/model/ds=20241014/model.tar.gz'

In [14]:
! aws s3 cp {s3_model_tar_file} ./data/

download: s3://warehouse-algo/rec_laidehe/test/all-ctr-test/model/ds=20241014/model.tar.gz to data/model.tar.gz


In [15]:
! cd data; tar -xzf model.tar.gz

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'


In [16]:
! pwd

/home/sagemaker-user


In [17]:
! rm ./data/model.tar.gz

In [18]:
! cd data; tar -czvf ./model.tar.gz ./*

./1730448714/
./1730448714/variables/
./1730448714/variables/variables.index
./1730448714/variables/variables.data-00000-of-00001
./1730448714/saved_model.pb
./code/
./code/inference.py
./goods_id2features.pkl


In [19]:
! aws s3 cp ./data/model.tar.gz {s3_model_tar_file}

upload: data/model.tar.gz to s3://warehouse-algo/rec_laidehe/test/all-ctr-test/model/ds=20241014/model.tar.gz


In [20]:
def ts2date(ts, fmt='%Y%m%d', offset=3600 * 8):
    return time.strftime(fmt, time.localtime(ts + offset))

def date2ts(date, fmt='%Y%m%d'):
    return time.mktime(time.strptime(date, fmt))

def alert(msg):
    print(msg)
    return
    msg = str(msg)
    max_size = 1000
    if len(msg) > max_size:
        msg = msg[:(max_size - 10)] + ' ...Over'
    sns_cli = boto3.client('sns')
    for phone_number in [13521039521, ]:
        sns_cli.publish(PhoneNumber='+86%s' % phone_number, Message=msg)

# alert('Hello India World!')

In [21]:
def get_s3_file_list_by_prefix(prefix, max_cnt=int(1e6)):
    '''
    return keys are not startswith bucket_prefix
    
    [{'Key': 'adm_search_sample_df_done/ds=20230225/part-00000-dee2825a858d-c000',
      'LastModified': datetime.datetime(2023, 2, 27, 9, 40, 21, tzinfo=tzlocal()),
      'ETag': '"c5fe35633015caa24e3b1b1487d89058"',
      'Size': 254,
      'StorageClass': 'STANDARD',
      'Owner': {'DisplayName': 'qingrong.zhao',
       'ID': '05d86fabdf3d5aa2a6de98d8d2abba9f3da5c6bfd675698131df15bb44b735c3'}},
     {'Key': ...}, ... ]
    '''
    file_list = sorted([v['Key'] for v in s3_cli.list_objects(Bucket=default_bucket,
        Prefix=prefix, MaxKeys=max_cnt).get('Contents', [])])
    print('List S3 %d files in prefix: %s' % (len(file_list), prefix))
    return file_list

def make_sure_dir(origin_path):
    if not origin_path.endswith('/') and '/' in origin_path:
        path = origin_path.rsplit('/', 1)[0]
    else:
        path = origin_path
    if not os.path.exists(path):
        try:
            os.makedirs(path) # multi-process make the same dir will cause dup error.
        except:
            pass
    return origin_path

def wait_for_ready(target_date, wait_window=3600 * 24 * 3, wait_interval=60 * 5):
    begin = time.time()
    model_data = model_data_pattern % target_date
    model_data_done = model_data + '.done'
    while True:
        now = time.time()
        if now - begin > wait_window:
            alert('Model of %s done %s DO NOT exists after %.2f hours, maybe something is wrong' % (
                target_date, model_data_done, (now - begin) / 3600))
            raise
        model_list = get_s3_file_list_by_prefix(model_data)
        if len(model_list) < 2:
            print('Model of %s done %s DO NOT exists, %d file(s), continue wait after %.2f hours' % (
                target_date, model_data_done, len(model_list), (now - begin) / 3600))
            time.sleep(wait_interval)
            continue
        elif len(model_list) > 2:
            alert('Too many Model files of %s, %d files, something is wrong' % (target_date, len(model_list)))
            raise
        elif model_list != [model_data, model_data_done]:
            alert('Model files of %s wrong: %s' % (target_date, model_list))
            raise
        
        print('Model of %s is ready now, start processing.' % target_date)
        return s3_bucket_prefix + model_data
    
def get_edp_status(edp_name, cur_date):
    status_file = make_sure_dir('sagemaker/edp_status/%s.status' % edp_name)
    s3_path = s3_bucket_prefix + status_file
    if not get_s3_file_list_by_prefix(status_file):
        return
    ! rm "{status_file}"
    ! aws s3 cp "{s3_path}" "{status_file}"
    with open(status_file, 'rt') as f:
        for line in f:
            return line.strip()

def set_edp_status(edp_name, cur_date):
    status_file = make_sure_dir('sagemaker/edp_status/%s.status' % edp_name)
    s3_path = s3_bucket_prefix + status_file
    with open(status_file, 'wt') as f:
        f.write(cur_date)
    ! aws s3 cp "{status_file}" "{s3_path}"
    ! rm "{status_file}"
    
def detect_need_update_or_not(edp_name, model_data_pattern):
    for day_gap in range(10):
        cur_date = ts2date(time.time() - 3600 * 24 * day_gap)
        model_data = model_data_pattern % cur_date
        model_data_done = model_data + '.done'
        model_list = get_s3_file_list_by_prefix(model_data)
        if len(model_list) < 2:
            print('Model of %s done %s DO NOT exists, %d file(s)' % (
                cur_date, model_data_done, len(model_list)))
            continue
        elif len(model_list) > 2:
            alert('Too many Model files of %s, %d files, something is wrong' % (cur_date, len(model_list)))
            raise
        elif model_list != [model_data, model_data_done]:
            alert('Model files of %s wrong: %s' % (cur_date, model_list))
            raise
        
        print('Latest Model of %s is ready now at date: %s.' % (model_data_pattern, cur_date))
        last_date = get_edp_status(edp_name, cur_date)
        print('Read status: ', last_date, ', cur date:', cur_date)
        if not last_date or last_date < cur_date:
            return cur_date
        break

def wait_edp_inservice(edp_name, wait_window=3600, wait_interval=10):
    begin = time.time()
    while True:
        now = time.time()
        if now - begin > wait_window:
            alert('Endpoint %s DO NOT inservice after %.2f hours, maybe something is wrong' % (
                edp_name, (now - begin) / 3600))
            raise
        edp_status = sm_cli.describe_endpoint(EndpointName=edp_name)
        if edp_status.get('EndpointStatus') != 'InService':
            print('Endpoint %s DO NOT inservice, continue wait after %.2f hours' % (
                edp_name, (now - begin) / 3600))
            time.sleep(wait_interval)
            continue
        
        print('Endpoint %s inservice now, finished.' % edp_name)
        break


In [28]:
def deploy_new_endpoint(model_data,
                        endpoint_name,
                        instance_type='ml.r5.large',
                        instance_count=1,
                        retry_times=0):
    # If an endpoint could describe, it exists, and can not be created by deploy.
    try:
        print(client.describe_endpoint(EndpointName=endpoint_name))
        return
    except:
        pass

    edp_model_name = endpoint_name + '-' + str(random.randint(10000, 19999))
    variant_name = "Variant-xlarge-1" # start from 1, incr 1 when updating.
    img = sagemaker.image_uris.retrieve(
        framework='tensorflow',
        version='1.15.2',
        region=sm_sess.boto_region_name,
        image_scope='inference',
        instance_type=instance_type
    )

    sm_sess.create_model(
        name=edp_model_name,
        role=role,
        container_defs={
            "Image": img,
            "ModelDataUrl": model_data,
            'Environment': {
                'TF_DISABLE_MKL': '1',
                'TF_DISABLE_POOL_ALLOCATOR': '1',
                # 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code/',  # Directory inside the container
                # 'SAGEMAKER_PROGRAM': 'inference.py', 
            },
        }
    )

    

    variant1 = production_variant(
        model_name=edp_model_name,
        instance_type=instance_type,
        initial_instance_count=instance_count,
        variant_name=variant_name,
        initial_weight=1,
    )

    sm_sess.endpoint_from_production_variants(
        name=endpoint_name, production_variants=[variant1],
        tags=[{'Key': 'cost-team', 'Value': 'algorithm'}],
    )
    print(sm_cli.describe_endpoint(EndpointName=endpoint_name))
    wait_edp_inservice(endpoint_name)


In [23]:
def update_model_data_of_endpoint(model,
                                  model_data,
                                  endpoint_name,
                                  instance_type='ml.r5.large',
                                  instance_count=1,
                                  retry_times=0):
    # If describe failed, stop updating.
    endpoint_info = sm_cli.describe_endpoint(EndpointName=endpoint_name)
    print(endpoint_info)
    cur_model = model.replace('_', '-')
    pre_var_name = endpoint_info['ProductionVariants'][0]['VariantName']    
    cur_version = str(retry_times + 1 + int(pre_var_name.split('-')[-1]))
    edp_model_name = '-'.join([endpoint_name, cur_model, cur_version])
    variant_name = "Variant-xlarge-%s-%s" % (cur_model, cur_version)
    endpoint_config_name = 'edp-config-%s-multi-variants-%s' % (cur_model, cur_version)
    
    print(cur_model, cur_version, pre_var_name, edp_model_name, variant_name,
          endpoint_config_name)
    # raise
    
    img = sagemaker.image_uris.retrieve(
        framework='tensorflow',
        version = '1.15.2',
        region = sm_sess.boto_region_name,
        image_scope='inference',
        instance_type = instance_type
    )
    
    sm_sess.create_model(
        name=edp_model_name,
        role=role,
        container_defs={
            "Image": img,
            "ModelDataUrl": model_data,
            'Environment': {
                'TF_DISABLE_MKL': '1',
                'TF_DISABLE_POOL_ALLOCATOR': '1',
                # 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code/',  # Directory inside the container
                # 'SAGEMAKER_PROGRAM': 'inference.py', 
            },
        }
    )
    
    variant1 = production_variant(
        model_name=edp_model_name,
        instance_type=instance_type,
        initial_instance_count=instance_count,
        variant_name=variant_name,
        initial_weight=1,
    )

    print(sm_cli.create_endpoint_config(
        EndpointConfigName = endpoint_config_name,
        ProductionVariants=[
            {
                "VariantName": variant_name,
                "ModelName": edp_model_name,
                "InstanceType": instance_type,
                "InitialInstanceCount": instance_count,
            },
        ]
    ))

    print(sm_cli.update_endpoint(
        EndpointName=endpoint_name,
        EndpointConfigName=endpoint_config_name,
    ))
    
    print(sm_cli.describe_endpoint(EndpointName=endpoint_name))
    wait_edp_inservice(endpoint_name)


In [24]:
def try_update_cur_model_by_config(update_config):
    model = update_config['model']
    endpoint_name = update_config['endpoint']
    model_data_pattern = update_config['model_data_pattern']
    
    # target_date = detect_need_update_or_not(endpoint_name, model_data_pattern)
    target_date = '20241014'
    print('Get date', model, target_date)
    if not target_date:
        return

    # continue
    ctx = Context()
    ctx.model = model
    ctx.endpoint_name = endpoint_name
    ctx.target_date = target_date
    ctx.model_data = s3_bucket_prefix + model_data_pattern % target_date
    steps = generate_steps()
    execute_steps_safely(ctx, steps)
    alert(ctx)
    ctx.dump()
    if not ctx.err_msg:
        set_edp_status(endpoint_name, target_date)

In [25]:
class Context:
    def __init__(self):
        self.model = None
        self.start_time = ts2date(time.time(), '%m-%d %H:%M:%S')
        self.start_date = ts2date(time.time()) # task started date
        self.target_date = ts2date(time.time() - 3600 * 24 * 3) # target sample date
        
        self.wait_window = 3600 * 24 * 3 # wait new samples ready at most 12 hours.
        self.wait_interval = 60 * 5 # check every 5 minutes
        
        self.model_data = None
        self.endpoint_name = None
        
        self.deploy_execute_msg = None
        self.update_execute_msg = None
        
        self.err_msg = None
        self.cost_desc_list = []
        
    def __str__(self):
        return '; '.join([
            '[%s] start at %s' % (ts2date(time.time(), '%m-%d %H:%M:%S'), self.start_time),
            'Model: %s' % self.model,
            '%s' % ('Succeeded' if not self.err_msg else ('Failed: %s' % self.err_msg)), 
            'target date: %s' % self.target_date,
            'Model data: %s' % self.model_data,
            'Deploy: %s' % self.deploy_execute_msg,
            'Update: %s' % self.update_execute_msg,
            'error msg: %s' % self.err_msg,
            'Over',
        ])
    
    def __repr__(self):
        return self.__str__()
    
    def dump(self, log_file=None):
        if log_file is None:
            log_file = make_sure_dir('log/%s/ctx_list.json' % self.model)
        with open(log_file, 'wt') as f:
            f.write(json.dumps(self.__dict__) + '\n')

            
def generate_steps():
    def wait_and_detect_step(ctx):
        ctx.model_data = wait_for_ready(ctx.target_date, ctx.wait_window, ctx.wait_interval)
        
    def deploy_if_need_step(ctx):
        try:
            sm_cli.describe_endpoint(EndpointName=ctx.endpoint_name)
            edp_exists = True
        except:
            edp_exists = False
        if not edp_exists:
            ctx.deploy_execute_msg = 'Deploy started'
            deploy_new_endpoint(ctx.model_data, ctx.endpoint_name)
            ctx.deploy_execute_msg = 'Deploy done'
        else:
            ctx.deploy_execute_msg = 'END already exists, DO NOT NEED to deploy.'
        
    def update_model_step(ctx):
        ctx.update_execute_msg = 'Update started'
        update_success = False
        for retry_times in range(20):
            try:
                update_model_data_of_endpoint(ctx.model, ctx.model_data,
                                              ctx.endpoint_name, retry_times=retry_times)
                update_success = True
                break
            except:
                traceback.print_exc()
                continue
        if not update_success:
            raise ValueError('Failed to update')
        ctx.update_execute_msg = 'Update done'

    return [
        # ['wait model update', wait_and_detect_step],
        ['try deploy model', deploy_if_need_step],
        ['update model of edp', update_model_step],
    ]

def execute_steps_safely(ctx, steps):
    for name, func in steps:
        begin = time.time()
        print('Begin execute step %s at %s' % (name, ts2date(begin, '%m-%d %H:%M:%S')))
        success = False
        try:
            func(ctx)
            success = True
        except Exception as e:
            traceback.print_exc()
        end = time.time()
        ctx.cost_desc_list.append('step [%s] begin at %s, cost %.2f hours' % (
            name, ts2date(begin, '%m-%d %H:%M:%S'), (end - begin) / 3600))
        if not success or ctx.err_msg:
            ctx.err_msg = 'Step [%s] FAILED for %s' % (name, ctx.err_msg)
            break


In [32]:
default_bucket = 'warehouse-algo'
s3_bucket_prefix = 's3://%s/' % default_bucket

model_update_config = [
    # {
    #     'model': 'ctr-v11',
    #     'endpoint': 'edp-ctr-v11',
    #     'model_data_pattern': 'sagemaker/search_new/all-ctr-v11/model/ds=%s/model.tar.gz',
    # },
    # s3://warehouse-algo/sagemaker/search_new/tf_model.tar.gz
    {
        'model': 'all-ctr-test',
        'endpoint': 'all-ctr-test-v3',
        'model_data_pattern': 'rec_laidehe/test/all-ctr-test/model/ds=%s/model.tar.gz',
    },
]

for update_config in model_update_config:
    try_update_cur_model_by_config(update_config)

Get date all-ctr-test 20241014
Begin execute step try deploy model at 11-18 17:18:12
Begin execute step update model of edp at 11-18 17:18:12
{'EndpointName': 'all-ctr-test-v3', 'EndpointArn': 'arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint/all-ctr-test-v3', 'EndpointConfigName': 'all-ctr-test-v3', 'ProductionVariants': [{'VariantName': 'Variant-xlarge-1', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference:1.15.2-cpu', 'ResolvedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference@sha256:c8ce28c662ff49396e546a7da6a30331df3b924a7d768888c8c5328abd7bdd2d', 'ResolutionTime': datetime.datetime(2024, 11, 18, 9, 12, 31, 893000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2024, 11, 18, 9, 12, 31, 393000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(20

Using already existing model: all-ctr-test-v3-all-ctr-test-2
Traceback (most recent call last):
  File "/tmp/ipykernel_30982/2618302691.py", line 65, in update_model_step
    update_model_data_of_endpoint(ctx.model, ctx.model_data,
  File "/tmp/ipykernel_30982/2275675785.py", line 52, in update_model_data_of_endpoint
    print(sm_cli.create_endpoint_config(
  File "/opt/conda/lib/python3.10/site-packages/botocore/client.py", line 569, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/opt/conda/lib/python3.10/site-packages/botocore/client.py", line 1023, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint-config/edp-config-all-ctr-test-multi-variants-2".


{'EndpointName': 'all-ctr-test-v3', 'EndpointArn': 'arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint/all-ctr-test-v3', 'EndpointConfigName': 'all-ctr-test-v3', 'ProductionVariants': [{'VariantName': 'Variant-xlarge-1', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference:1.15.2-cpu', 'ResolvedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference@sha256:c8ce28c662ff49396e546a7da6a30331df3b924a7d768888c8c5328abd7bdd2d', 'ResolutionTime': datetime.datetime(2024, 11, 18, 9, 12, 31, 893000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2024, 11, 18, 9, 12, 31, 393000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 11, 18, 9, 14, 45, 95000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'c76ac876-ff95-48af-af5e-cb4de5bb276a', 'HTTPStatusCode': 2

Using already existing model: all-ctr-test-v3-all-ctr-test-3
Traceback (most recent call last):
  File "/tmp/ipykernel_30982/2618302691.py", line 65, in update_model_step
    update_model_data_of_endpoint(ctx.model, ctx.model_data,
  File "/tmp/ipykernel_30982/2275675785.py", line 52, in update_model_data_of_endpoint
    print(sm_cli.create_endpoint_config(
  File "/opt/conda/lib/python3.10/site-packages/botocore/client.py", line 569, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/opt/conda/lib/python3.10/site-packages/botocore/client.py", line 1023, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint-config/edp-config-all-ctr-test-multi-variants-3".


{'EndpointName': 'all-ctr-test-v3', 'EndpointArn': 'arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint/all-ctr-test-v3', 'EndpointConfigName': 'all-ctr-test-v3', 'ProductionVariants': [{'VariantName': 'Variant-xlarge-1', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference:1.15.2-cpu', 'ResolvedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference@sha256:c8ce28c662ff49396e546a7da6a30331df3b924a7d768888c8c5328abd7bdd2d', 'ResolutionTime': datetime.datetime(2024, 11, 18, 9, 12, 31, 893000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2024, 11, 18, 9, 12, 31, 393000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 11, 18, 9, 14, 45, 95000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '6ce5e074-a3f7-4819-9d9e-d83e204bb216', 'HTTPStatusCode': 2

...

Using already existing model: all-ctr-test-v3-all-ctr-test-20
Traceback (most recent call last):
  File "/tmp/ipykernel_30982/2618302691.py", line 65, in update_model_step
    update_model_data_of_endpoint(ctx.model, ctx.model_data,
  File "/tmp/ipykernel_30982/2275675785.py", line 52, in update_model_data_of_endpoint
    print(sm_cli.create_endpoint_config(
  File "/opt/conda/lib/python3.10/site-packages/botocore/client.py", line 569, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/opt/conda/lib/python3.10/site-packages/botocore/client.py", line 1023, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint-config/edp-config-all-ctr-test-multi-variants-20".


{'EndpointName': 'all-ctr-test-v3', 'EndpointArn': 'arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint/all-ctr-test-v3', 'EndpointConfigName': 'all-ctr-test-v3', 'ProductionVariants': [{'VariantName': 'Variant-xlarge-1', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference:1.15.2-cpu', 'ResolvedImage': '763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/tensorflow-inference@sha256:c8ce28c662ff49396e546a7da6a30331df3b924a7d768888c8c5328abd7bdd2d', 'ResolutionTime': datetime.datetime(2024, 11, 18, 9, 12, 31, 893000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2024, 11, 18, 9, 12, 31, 393000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 11, 18, 9, 14, 45, 95000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '09e593ad-335c-4981-8101-b97b1f9fe841', 'HTTPStatusCode': 2

Using already existing model: all-ctr-test-v3-all-ctr-test-21


[11-18 17:18:34] start at 11-18 17:18:12; Model: all-ctr-test; Failed: Step [update model of edp] FAILED for None; target date: 20241014; Model data: s3://warehouse-algo/rec_laidehe/test/all-ctr-test/model/ds=20241014/model.tar.gz; Deploy: END already exists, DO NOT NEED to deploy.; Update: Update started; error msg: Step [update model of edp] FAILED for None; Over


Traceback (most recent call last):
  File "/tmp/ipykernel_30982/2618302691.py", line 65, in update_model_step
    update_model_data_of_endpoint(ctx.model, ctx.model_data,
  File "/tmp/ipykernel_30982/2275675785.py", line 52, in update_model_data_of_endpoint
    print(sm_cli.create_endpoint_config(
  File "/opt/conda/lib/python3.10/site-packages/botocore/client.py", line 569, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/opt/conda/lib/python3.10/site-packages/botocore/client.py", line 1023, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:ap-southeast-1:580243821396:endpoint-config/edp-config-all-ctr-test-multi-variants-21".
Traceback (most recent call last):
  File "/tmp/ipykernel_30982/2618302691.py", line 88, in execute_steps_safely
    f